In [1]:
import pandas as pd

In [202]:
scf = pd.read_stata('scf86b.dta')

In [203]:
# SMSA, FIPS, reasons for saving (18-20), total amount of non-residence assets, total private business assets
scf = scf[['v1', 'c1211', 'c1213', 'c1218', 'c1219', 'c1220', 'c1417', 'c1419' ]]

In [195]:
scf.head()

,c1211,c1213,c1218,c1219,c1220,c1417,c1419
0,6760,51760,-6,0,0,0,0
1,1140,12081,-6,0,0,0,0
2,7360,6075,26,0,0,0,0
3,1140,12081,22,3,24,1774,0
4,6760,51760,-6,0,0,0,0


In [103]:
crossover = pd.read_csv('counties.csv')

In [104]:
crossover.head()

,county
0,"1073. Jefferson County, AL (2 cases)"
1,"1101. Montgomery County, AL (20 cases)"
2,"2110. Juneau County, AK (1 case)"
3,"2122. Kenai Peninsula County, AK (1 case)"
4,"4013. Maricopa County, AZ (22 cases)"


In [105]:
crossover[['fips', 'cname', 'none']] = crossover['county'].str.split('.', expand=True)

In [107]:
crossover[['cname', 'case']] = crossover['cname'].str.split('(', expand=True)

In [108]:
crossover['cname']  = crossover['cname'].str.replace(', ', '').str.replace('County', '')

In [110]:
crossover = crossover[['cname', 'fips']]
crossover.to_csv('crossover.csv')

In [177]:
new_crossover = pd.read_csv('crossover.csv')

In [178]:
new_crossover[['cname', 'state_ab', 'none']] = new_crossover['cname'].str.split(r'( [A-Z]{2})', expand=True)

In [179]:
new_crossover = new_crossover[['State', 'cname', 'fips']]

In [180]:
new_crossover = new_crossover.rename(columns={'State':'STAB', 'cname':'CNAME SR'})

In [185]:
new_crossover['CNAME SR'] = new_crossover['CNAME SR'].str.upper().str.strip(' ')
new_crossover['STAB'] = new_crossover['STAB'].str.strip(' ')

In [162]:
fips = pd.read_csv('new_fishback.csv')

In [163]:
fips.head()

,STAB,CNAME SR,RLDF3929,RLDF3329,RLDF3529
0,ALABAMA,AUTAUGA,-0.018973,-0.502064,-0.247065
1,ALABAMA,BALDWIN,0.187411,-0.497282,-0.204761
2,ALABAMA,BARBOUR,-0.014156,-0.468473,-0.081819
3,ALABAMA,BIBB,-0.124208,-0.603432,-0.476398
4,ALABAMA,BLOUNT,0.150613,-0.614978,-0.233145


In [186]:
new_crossover.head()

,STAB,CNAME SR,fips
0,ALABAMA,JEFFERSON,1073
1,ALABAMA,MONTGOMERY,1101
2,ALASKA,JUNEAU,2110
3,ALASKA,KENAI PENINSULA,2122
4,ARIZONA,MARICOPA,4013


In [187]:
fips.head()

,STAB,CNAME SR,RLDF3929,RLDF3329,RLDF3529
0,ALABAMA,AUTAUGA,-0.018973,-0.502064,-0.247065
1,ALABAMA,BALDWIN,0.187411,-0.497282,-0.204761
2,ALABAMA,BARBOUR,-0.014156,-0.468473,-0.081819
3,ALABAMA,BIBB,-0.124208,-0.603432,-0.476398
4,ALABAMA,BLOUNT,0.150613,-0.614978,-0.233145


In [189]:
fips_retail = fips.merge(new_crossover, on=['STAB', 'CNAME SR'])

In [219]:
scf = scf.rename(columns={'c1213':'fips'})

In [247]:
final_data = scf.merge(fips_retail, on=['fips'])

In [248]:
final_data.head()

,v1,c1211,fips,c1218,c1219,c1220,c1417,c1419,STAB,CNAME SR,RLDF3929,RLDF3329,RLDF3529
0,1,6760,51760,-6,0,0,0,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548
1,5,6760,51760,-6,0,0,0,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548
2,121,6760,51760,22,0,0,78000,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548
3,506,6760,51760,24,0,0,0,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548
4,955,6760,51760,25,0,0,0,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548


In [249]:
# Top three reasons for saving
# Choosen options to proxy: invest in business, invest in general, wise to save, invest in own education
final_data['c1218_map'] = final_data['c1218'].map({21: 1, 26: 1, 2: 1, 91: 1}).fillna(0)
final_data['c1219_map'] = final_data['c1219'].map({21: 1, 26: 1, 2: 1, 91: 1}).fillna(0)
final_data['c1220_map'] = final_data['c1220'].map({21: 1, 26: 1, 2: 1, 91: 1}).fillna(0)
final_data['sav_tot'] = final_data['c1218_map'] + final_data['c1219_map'] + final_data['c1220_map']

In [250]:
final_data['c1218_map'].value_counts()

0.0    2082
1.0      80
Name: c1218_map, dtype: int64

In [251]:
final_data['c1219_map'].value_counts()

0.0    2090
1.0      72
Name: c1219_map, dtype: int64

In [252]:
final_data['c1220_map'].value_counts()

0.0    2134
1.0      28
Name: c1220_map, dtype: int64

In [253]:
final_data['sav_tot'].value_counts()

0.0    1984
1.0     176
2.0       2
Name: sav_tot, dtype: int64

In [254]:
#Real Estate Holdings - not primary residence
final_data['c1417'].value_counts().head(15)

0         1673
35000       22
50000       18
100000      18
5000        17
60000       16
20000       15
30000       13
10000       13
15000       12
150000      10
12000       10
75000        9
40000        9
200000       9
Name: c1417, dtype: int64

In [255]:
#Privately held business assets
final_data['c1419'].value_counts().head(15)

 0         1869
 1000        22
 50000       10
 10000        8
 200000       8
 100000       5
 2000         5
 150000       5
 5000         5
 250000       4
-6            4
 40000        3
 60000        3
 35000        3
 20000        3
Name: c1419, dtype: int64

In [257]:
final_data.head()

,v1,c1211,fips,c1218,c1219,c1220,c1417,c1419,STAB,CNAME SR,RLDF3929,RLDF3329,RLDF3529,c1218_map,c1219_map,c1220_map,sav_tot
0,1,6760,51760,-6,0,0,0,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548,0.0,0.0,0.0,0.0
1,5,6760,51760,-6,0,0,0,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548,0.0,0.0,0.0,0.0
2,121,6760,51760,22,0,0,78000,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548,0.0,0.0,0.0,0.0
3,506,6760,51760,24,0,0,0,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548,0.0,0.0,0.0,0.0
4,955,6760,51760,25,0,0,0,0,VIRGINIA,RICHMOND,0.460085,-0.02696,0.092548,0.0,0.0,0.0,0.0


In [258]:
final_dataset = final_data[['c1218_map', 'c1219_map', 'c1220_map', 'sav_tot', 'c1417', 'c1419', 'RLDF3929', 'RLDF3529', 'RLDF3329']]

In [259]:
final_dataset.to_csv('final_regression_data.csv')